In [198]:
import pandas as pd
df_train = pd.read_csv('./source/datasets/df_treino.csv')
df_test = pd.read_csv('./source/datasets/df_teste.csv')


In [199]:
print(df_train['Resultado'].value_counts())
print(df_test['Resultado'].value_counts())

0    5626
1     158
Name: Resultado, dtype: int64
0    1409
1      32
Name: Resultado, dtype: int64


In [200]:
y_train = df_train['Resultado'].to_numpy()
X_train = df_train.drop(['Resultado'], axis=1).to_numpy()

In [201]:
y_train_save = y_train
X_train_save = X_train

## 1) Balanceando as classes

Primeiramente, precisamos garantir representatividade de ao menos 1,5x da classe majoritária sobre a minoritária. Como temos consideravelmente mais instâncias de pessoas que não faleceram, vamos buscar utilizar diferentes técnicas tanto para selecionar instâncias da classe majoritária quanto para aumentar as instâncias da classe majoritária

Para isso, iremos utilizar a biblioteca imblearn

In [202]:
# !pip install imblearn

Primeiramente, aumentar a instância da classe minoritaria até atingir 20% da majoritária

In [203]:
from imblearn.over_sampling import SMOTE
import numpy as np

oversample = SMOTE(sampling_strategy=0.2, random_state=42)
print("Quantidade da classe minoritária antes da aumentação: ", np.unique(y_train, return_counts=True))

X_train, y_train = oversample.fit_resample(X_train, y_train)
print("Quantidade da classe minoritária após aumentação: ", np.unique(y_train, return_counts=True))

Quantidade da classe minoritária antes da aumentação:  (array([0, 1]), array([5626,  158]))
Quantidade da classe minoritária após aumentação:  (array([0, 1]), array([5626, 1125]))


Agora, vamos utilizar undersampling para reduzir a classe majoritária. 

In [204]:
from imblearn.under_sampling import NearMiss

nm = NearMiss(sampling_strategy={0:1687,1:1125})
X_train, y_train = nm.fit_resample(X_train, y_train)
print(X_train.shape)
print(y_train.shape)

(2812, 20)
(2812,)


In [205]:
import numpy as np
print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([1687, 1125]))


## 2) Treinamento do modelo

A análise anterior foi usada apenas para demonstrar como devem ser balanceados os modelos. O balanceamento será realizado dentro dos folds de treinamento para cada iteração do k-fold, a fim de que a valicação reflita melhor os padrões que serão vistos nos conjuntos de teste

In [206]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline as SKPipeline
from sklearn.metrics import classification_report

X_train = X_train_save
y_train = y_train_save

print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([5626,  158]))


In [219]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

param_grids = {
    'kNN': {
        'n_neighbors': [1, 5, 10],
        'p': [1, 2],
        'weights': ['uniform', 'distance']
    },
    'Logistic Regression': {
        'class_weight': ['balanced', {2: 1, 3: 1}]
    },
    'Random Forest': {
        'n_estimators': [10, 100, 200],
        'max_depth': [10, 50],
        'min_samples_split': [2, 10, 30]
    }
}


Agora que já temos os parâmetros iniciais, instanciando os modelos:

In [220]:
# Modelos base
knn = KNeighborsClassifier()
lr = LogisticRegression()
rf = RandomForestClassifier(random_state=42)

models = [('kNN', knn), ('Logistic Regression', lr), ('Random Forest', rf)]

In [223]:
from sklearn.base import clone
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, base_model in models:
    
    all_reports = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        print(f'Fold {fold+1}')
        X_train_fold, X_val = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val = y_train[train_idx], y_train[val_idx]

        # Transformando classe minoritária em 20% da classe majoritária
        oversample = SMOTE(sampling_strategy=0.2, random_state=42)
        X_train_fold, y_train_fold = oversample.fit_resample(X_train_fold, y_train_fold)

        # Transformando classe majoritária em 1.5x da minoritária
        nm = NearMiss(sampling_strategy=0.6)
        X_train_fold, y_train_fold = nm.fit_resample(X_train_fold, y_train_fold)

        # Mostrando a distribuição do y treino, para ver se o 60% foi respeitado
        print("Distribuição y treino: ", np.unique(y_train_fold, return_counts=True))

        # Clonando o modelo
        model = clone(base_model)

        # Definindo o GridSearch
        grid = GridSearchCV(
            model,
            param_grid=param_grids[name],
            scoring={'recall': 'recall', 'precision': 'precision'},
            refit='recall',  # Use 'recall' to refit the model
            cv=5,
            n_jobs=-1)

        # Ajustando o modelo com os dados balanceados
        grid.fit(X_train_fold, y_train_fold)

        # Usando o melhor modelo
        best_model = grid.best_estimator_

        # Avaliando o modelo
        y_pred = best_model.predict(X_val)

        # Gerando o classification report
        report = classification_report(y_val, y_pred, output_dict=True)
        all_reports.append(report)  





    

Fold 1
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 2
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 3
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 4
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 5
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 1
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 2


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 3


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 4
Distribuição y treino:  (array([0, 1]), array([1500,  900]))


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

Fold 5
Distribuição y treino:  (array([0, 1]), array([1500,  900]))


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

Fold 1
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 2
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 3
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 4
Distribuição y treino:  (array([0, 1]), array([1500,  900]))
Fold 5
Distribuição y treino:  (array([0, 1]), array([1500,  900]))


KeyboardInterrupt: 